In [7]:
import os
import pandas as pd
from bertopic import BERTopic

In [2]:
data_path = '../CRAFT/data/'

In [11]:
ontologies = os.listdir(data_path)
ontologies = [o for o in ontologies if os.path.isdir(os.path.join(data_path, o))]

In [53]:
dfs = []
for o in ontologies:
    dfs.append(pd.read_csv(os.path.join(data_path, o, "annotations.csv"))[['PMID', 'MentionTextSegment']])
mentions = pd.concat(dfs)
mentions = mentions.groupby('PMID').apply(lambda x: (list(x.MentionTextSegment)))

In [54]:
mentions

PMID
11319941    [neuron, neuron, neuron, neuron, neuron, neuro...
11532192    [biological pigment, aging, circadian rhythm, ...
11597317    [double-strand break repair via homologous rec...
11604102    [gene expression, gene expression, gene expres...
11897010    [cation, ion, alternative mRNA splicing, via s...
                                  ...                        
17565376    [T cell, M cell of gut, Peyer's patch developm...
17590087    [Purkinje cell, gene expression, autosome, aut...
17608565    [acetylcholine, neuron, neuron, amacrine cell,...
17677002    [polyatomic entity, polyatomic entity, neuron,...
17696610    [okadaic acid, spermatocyte, oocyte, spermatoc...
Length: 97, dtype: object

In [55]:
## Training

In [56]:
def transform_strings_to_arrays(df, col_names = ['tokenized_sentences', 'tokenized_words', 'tokenized_words_processed', 'tokenized_words_no_stopwords', 'tokenized_words_lemmatize']):
    for col in col_names:
        df[col] = df[col].apply(eval)
    return df

In [ ]:
# basic BertTopic keyword extraction
def train_transform(train_data, model_save_name, min_topic_size=10):
    
    # train transform
    topic_model = BERTopic(min_topic_size=min_topic_size)
    topics, probs = topic_model.fit_transform(train_data.values)

    return topic_model, topics, probs

In [ ]:
train_full_data = transform_strings_to_arrays(pd.read_csv(train_data_path))

train_data = train_full_data.groupby(by = ['PMID'])['tokenized_words_processed'].agg(lambda x: ' '.join(x.values[0] + x.values[1]))

topic_model, topics, probs = train_transform(train_data)